<a href="https://colab.research.google.com/github/maski07/machine-learning/blob/main/natural_speech_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 事前準備



In [ ]:
!pip install naturalspeech2-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 36.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 73.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparin

In [ ]:
import torch
from naturalspeech2_pytorch import (
    EncodecWrapper,
    Model,
    NaturalSpeech2,
    SpeechPromptEncoder
)

In [ ]:
# use encodec as an example
codec = EncodecWrapper()

prompt_encoder = SpeechPromptEncoder(
    dim_codebook = codec.codebook_dim,
    depth = 2
)

model = Model(
    dim = 128,
    depth = 6
)

# natural speech diffusion model
diffusion = NaturalSpeech2(
    model = model,
    codec = codec,
    timesteps = 1000
).cuda()

## 音声の声色を学習

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from naturalspeech2_pytorch import Trainer

trainer = Trainer(
    diffusion_model = diffusion,     # diffusion model + codec from above
#    folder = '/content/drive/MyDrive/program/',
    folder = '/path/to/speech',
    # results_folder = '/content/drive/MyDrive/program/',
    #  results_folder = '/result',
    train_batch_size = 16,
    gradient_accumulate_every = 2,
)

trainer.train()

  0%|          | 0/100000 [00:00<?, ?it/s]

RuntimeError: ignored

### 音声の出力

In [ ]:
text = "「天は人の上に人を造らず人の下に人を造らず」と言えり。されば天より人を生ずるには、万人は万人みな同じ位にして、生まれながら貴賤きせん上下の差別なく、万物の霊たる身と心との働きをもって天地の間にあるよろずの物を資とり、もって衣食住の用を達し、自由自在、互いに人の妨げをなさずしておのおの安楽にこの世を渡らしめ給うの趣意なり。されども今、広くこの人間世界を見渡すに、かしこき人あり、おろかなる人あり、貧しきもあり、富めるもあり、貴人もあり、下人もありて、その有様雲と泥どろとの相違あるに似たるはなんぞや。その次第はなはだ明らかなり。『実語教じつごきょう』に、「人学ばざれば智なし、智なき者は愚人なり」とあり。されば賢人と愚人との別は学ぶと学ばざるとによりてできるものなり。また世の中にむずかしき仕事もあり、やすき仕事もあり。そのむずかしき仕事をする者を身分重き人と名づけ、やすき仕事をする者を身分軽き人という。すべて心を用い、心配する仕事はむずかしくして、手足を用うる力役りきえきはやすし。ゆえに医者、学者、政府の役人、または大なる商売をする町人、あまたの奉公人を召し使う大百姓などは、身分重くして貴き者と言うべし。"

In [ ]:
text="Hello, I’d like to check in. My name’s Taro Yamada."

In [ ]:
audio = trainer.print(text)

In [ ]:
import torchaudio
from IPython.display import Audio

# audio を numpy 配列に変換
audio_numpy = audio.cpu().squeeze().numpy()

# オーディオを再生
Audio(audio_numpy, rate=16000)  # rateはサンプリングレートに応じて調整してください

### エラー調査

Traner.train()と同じところで落ちることを確認できる。

In [ ]:
# mock raw audio data

raw_audio = torch.randn(4, 327680).cuda()
prompt = torch.randn(4, 32768).cuda()          # they randomly excised a range on the audio for the prompt during training, eventually will take care of this auto-magically

## loss = diffusion(raw_audio, prompt = prompt) # pass in the prompt
loss = diffusion(raw_audio) # pass in the prompt

loss.backward()

AssertionError: ignored

### メモ

In [ ]:
import shutil
import os

def delete_files_in_directory(directory_path):
    # 指定したディレクトリ内のファイルのリストを取得
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

    # 各ファイルを削除
    for file in files:
        os.remove(os.path.join(directory_path, file))

# 作成したいフォルダのパス
path = "/path/to/speech"

# フォルダが存在すればフォルダをクリア、存在しない場合フォルダ作成
if os.path.exists(path):
  delete_files_in_directory(path)
else:
  os.makedirs(path)

# ファイル名
fine_names = ['sample.mp3', 'sample_20s_woman.mp3','sample_20s_man.mp3', 'sample_5s_man_en.mp3']
file_name = fine_names[3]

# 移動元のファイルパス
src_path = '/content/drive/MyDrive/program/' + file_name

# 移動先のファイルパス
dst_path = path + '/' + file_name

# ファイルをコピー
shutil.copyfile(src_path, dst_path)

'/path/to/speech/sample_5s_man_en.mp3'